In [0]:
import tensorflow as tf
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator  
from keras.applications import densenet  
from keras.models import Sequential, Model, load_model  
from keras.optimizers import Adam, SGD
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Input, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense  
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback  
from keras import regularizers  
from keras import backend as K 
from PIL import Image
import numpy as np

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet_v2 import ResNet50V2

import matplotlib.pyplot as plt

tf.__version__

'1.15.0'

In [0]:
def rmse(y_pred, y_true):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [0]:
def pred_rmse(y_pred, y_true):
  return np.sqrt(np.mean((y_pred.values-y_true.squeeze())**2))

In [0]:
percentages = pd.read_csv('/content/drive/My Drive/Master/DeepLearning/UNIMIB2016/percentage.csv')

In [0]:
test_index = int(len(percentages)*0.75)
train = percentages.iloc[:test_index,:]
test = percentages.iloc[test_index:,:]

In [0]:
validation_split = 0.1
train_data_dir = '/content/drive/My Drive/Master/DeepLearning/UNIMIB2016/images'
img_height, img_width = 640, 480
batch_size = 32
nb_epochs = 100

train_datagen = ImageDataGenerator(rescale=1./255,
    validation_split=validation_split, horizontal_flip=True,
     vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0])

test_datagen = ImageDataGenerator(rescale=1./255)  

train_generator = train_datagen.flow_from_dataframe(
     dataframe=train,
    directory=train_data_dir,
    x_col="image_name",
    y_col="food_pixels",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="raw",
    #color_mode = "grayscale",
    subset='training')

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory=train_data_dir,
    x_col="image_name",
    y_col="food_pixels",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="raw",
    #color_mode = "grayscale",
    subset='validation')

test_generator = test_datagen.flow_from_dataframe(
     dataframe=test,
    directory=train_data_dir,
    x_col="image_name",
    y_col="food_pixels",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="raw",shuffle=False)

Found 693 validated image filenames.
Found 77 validated image filenames.
Found 257 validated image filenames.


In [0]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_height, img_width, 3)))
for i, layer in enumerate(base_model.layers[1:]):
  layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(32)(x)
x = Activation("relu")(x)
x = BatchNormalization(axis=-1)(x)
x = Dropout(0.3)(x)

x = Dense(16)(x)
x = Activation("relu")(x)

predictions = Dense(1, init='glorot_uniform', activation="linear")(x)

model = Model(input=base_model.input, output=predictions)

opt = Adam(lr=1e-3)

es = EarlyStopping(monitor='val_loss', patience=10, mode='auto', restore_best_weights=True)
r_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, mode='auto',  min_lr=1e-6)
model.compile(loss="mean_squared_error", optimizer=opt, metrics=['mae', 'mse', rmse])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="linear", kernel_initializer="glorot_uniform")`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 640, 480, 3)  0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 319, 239, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_191 (BatchN (None, 319, 239, 32) 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_193 (Activation)     (None, 319, 239, 32) 0           batch_normalization_191[0][0]    
____________________________________________________________________________________________

In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs, callbacks = [es, r_lr])

Epoch 1/100
21/21 [==============================] - 68s 3s/step - loss: 223.2199 - mean_absolute_error: 14.0989 - mean_squared_error: 223.2199 - rmse: 14.9026 - val_loss: 181.6114 - val_mean_absolute_error: 12.4655 - val_mean_squared_error: 181.6114 - val_rmse: 13.4302
Epoch 2/100
21/21 [==============================] - 64s 3s/step - loss: 192.3011 - mean_absolute_error: 13.0113 - mean_squared_error: 192.3011 - rmse: 13.8336 - val_loss: 182.6724 - val_mean_absolute_error: 12.2328 - val_mean_squared_error: 182.6724 - val_rmse: 13.5133
Epoch 3/100
21/21 [==============================] - 62s 3s/step - loss: 173.0251 - mean_absolute_error: 12.2690 - mean_squared_error: 173.0251 - rmse: 13.1097 - val_loss: 160.1967 - val_mean_absolute_error: 11.9246 - val_mean_squared_error: 160.1967 - val_rmse: 12.6523
Epoch 4/100
21/21 [==============================] - 64s 3s/step - loss: 152.9774 - mean_absolute_error: 11.4515 - mean_squared_error: 152.9774 - rmse: 12.3259 - val_loss: 167.8149 - val_

In [0]:
import numpy as np
print("RMSE: "+str(pred_rmse(test["food_pixels"], model.predict(test_generator))))

RMSE: 2.298025888254974


In [0]:
model.save('/content/drive/My Drive/Master/DeepLearning/UNIMIB2016/models/inception_v3_transfer_model.h5')